In [15]:
import os
import numpy as np
import cv2  
import glob 
from matplotlib import pyplot as plt
import joblib

In [16]:
#Changing Current Working Directory;
path = r"E:\Dataset\Alzheimer_s Dataset"
def change_dir(path):
    retval = os.getcwd()    # Check current working directory.
    print("Current working directory ", retval) 
    os.chdir( path )   # Now change the directory.
    retval = os.getcwd() # Check current working directory.
    print ("Directory changed successfully ",retval) 
change_dir(path)

Current working directory  E:\Dataset\Alzheimer_s Dataset
Directory changed successfully  E:\Dataset\Alzheimer_s Dataset


In [17]:
#saving model 
def save_mod(model,Model_name):
    change_dir(r"C:\Users\iamne\JupyterNotebookServer\dev\ProjAlzhiemer")
    Model_name+='.sav'
    joblib.dump(model,Model_name)

In [18]:
# reading images from directory

def load_image(path):
    lst=[]
    for image_path in glob.glob(path):
        img = cv2.imread(image_path,0)
        img = img.reshape(1,36608)
        lst.append(img[0])
    return lst
    
temp = path + r"\train\MildDemented\*.jpg"
Mild_train = load_image(temp)
print("Mild = ",len(Mild_train))
temp = path + r"\train\ModerateDemented\*.jpg"
Mod_train = load_image(temp)
print("Mod = ",len(Mod_train))
temp = path + r"\train\NonDemented\*.jpg"
Non_train = load_image(temp)
print("Non = ",len(Non_train))
temp = path + r"\train\VeryMildDemented\*.jpg"
VMild_train = load_image(temp)
print("VMild = ",len(VMild_train))

Mild =  717
Mod =  52
Non =  2560
VMild =  1792


In [19]:

    
temp = path + r"\test\MildDemented\*.jpg"
Mild_test = load_image(temp)
print("Mild = ",len(Mild_test))
temp = path + r"\test\ModerateDemented\*.jpg"
Mod_test = load_image(temp)
print("Mod = ",len(Mod_test))
temp = path + r"\test\NonDemented\*.jpg"
Non_test = load_image(temp)
print("Non = ",len(Non_test))
temp = path + r"\test\VeryMildDemented\*.jpg"
VMild_test = load_image(temp)
print("VMild = ",len(VMild_test))

Mild =  179
Mod =  12
Non =  640
VMild =  448


In [20]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [21]:
train_x = Mild_train+VMild_train+Mod_train+Non_train
train_y = []

def ohe_ml(*lst):
    train_y = []
    en =1
    for j in range(0,len(lst)):
        for i in range(0,len(lst[j])):
            train_y.append(en)
        en*=10
    return train_y
train_y= ohe_ml( Mild_train,VMild_train,Mod_train,Non_train)

In [22]:
test_x = Mild_test+VMild_test+Mod_test+Non_test
test_y = []

def ohe_ml(*lst):
    train_y = []
    en =1
    for j in range(0,len(lst)):
        for i in range(0,len(lst[j])):
            train_y.append(en)
        en*=10
    return train_y
test_y= ohe_ml( Mild_test,VMild_test,Mod_test,Non_test)

In [23]:
print("train\n")
print(len(train_x),len(train_y))
print(train_x[0].shape)
loc = 732
print(train_x[loc],train_y[loc])
#--------------------------------------
print("test\n")
print(len(test_x),len(test_y))
print(test_x[0].shape)
loc = 340
print(test_x[loc],test_y[loc])

train

5121 5121
(36608,)
[0 0 0 ... 0 0 0] 10
test

1279 1279
(36608,)
[0 0 0 ... 0 0 0] 10


In [24]:
models = list()

decision_tree = Pipeline([('m', DecisionTreeClassifier())])
models.append(('decision', decision_tree))

randomforest = Pipeline([('m', RandomForestClassifier())])
models.append(('randomforest', randomforest))

svc = Pipeline([('m', SVC())])
models.append(('svc', svc))
# define the voting ensemble
lr = LogisticRegression()
stack_model = StackingClassifier( estimators = models,final_estimator = lr)

In [25]:
stack_model

StackingClassifier(estimators=[('decision',
                                Pipeline(steps=[('m',
                                                 DecisionTreeClassifier())])),
                               ('randomforest',
                                Pipeline(steps=[('m',
                                                 RandomForestClassifier())])),
                               ('svc', Pipeline(steps=[('m', SVC())]))],
                   final_estimator=LogisticRegression())

In [26]:
models

[('decision', Pipeline(steps=[('m', DecisionTreeClassifier())])),
 ('randomforest', Pipeline(steps=[('m', RandomForestClassifier())])),
 ('svc', Pipeline(steps=[('m', SVC())]))]

In [ ]:
ens = stack_model.fit(train_x,train_y)

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
n_scores = cross_val_score(ens, test_x, test_y, scoring='accuracy', cv=cv, n_jobs=-1)

In [ ]:
mean = np.mean(n_scores)
print("mean = ", mean)

In [ ]:
ser=ens.score(train_x,train_y)
print(ser)

In [ ]:
#save_mod(e,"test_tf_df")